In [ ]:
import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
#import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex
import emoji #pip install emoji --upgrade
# Internal dependencies
#import word_emoji2vec as we2v
#from word_emoji2vec import Word_Emoji2Vec

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

import json
import pickle

###word cloud
#!pip install wordcloud
from wordcloud import WordCloud 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import os
from datetime import datetime
import argparse
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch.distributed as dist
from apex.parallel import DistributedDataParallel as DDP 
from apex import amp
#os.environ['CUDA_VISIBLE_DEVICES'] = '1, 2, 3' 
#....to install apex...
#pip install -v --no-cache-dir ./

nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed #python -m spacy download en

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = False
np.random.seed(1)

# torch.manual_seed(1)
# np.random.seed(1)


In [ ]:
import numpy as np, pandas as pd 
import os
import zipfile

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy import stats

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [ ]:

df = pd.read_csv('data/keto/graph_embd_keto/output/des_only/user_des.embeddings', sep="\t", names=["user", "vector"], index_col=0)
input_dic = {key: val.values for key, val in df.T.items()}

In [ ]:
##split vectors based on space and convert to list
split_vector = []
for i in range (0, df.shape[0]):
    split_vector.append(df.vector[i].split(' '))
    
    

In [ ]:
#len(split_vector), split_vector[0]
split_vector[0][0], type(split_vector[0][0])

In [ ]:
#convert an array of strings to an array of floats in numpy?

np_split_vector = np.array(split_vector)
emb = np_split_vector.astype(np.float)

In [ ]:
type(emb[0][0])

In [ ]:
len(emb)

In [ ]:
user_dic = {}

for i in range (0,len(emb)):
    for k in input_dic:
        if k not in user_dic:
            user_dic[k] = emb[i]

In [ ]:
##load the predicted data
predicted_df = pd.read_csv( 'data/keto/predicted_keto_14k.csv',low_memory=False) #(28189, 6)


In [ ]:

df_q = predicted_df[['name',  'utype_gt']] #extracted 2 columns
#print(df_q)
df_q1= df_q.dropna()#dropping nan value
#print(df_q1) #1300
df_q1 = df_q1[df_q1.utype_gt != 2.0] #dropping rows with truth value 2.0
df_q1 = df_q1.reset_index(drop = True)
print(df_q1) #908


In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

#     for word in model.wv.vocab:
#         tokens.append(model[word])
#         labels.append(word)
#     for k, v in user_dic.items():
#         if k in df_q1.name:
#             tokens.append(v)
#             #labels.append(k)
#             labels.append(df_q1.utype_gt[])
        
    for k, v in user_dic.items():
        if k in df_q1.name.values:
            #print(k)
            tokens.append(v)
            #labels.append(df_q1.loc[(df_q1['name'] == k)].utype_gt.values)
            lb = (df_q1.loc[(df_q1['name'] == k)].utype_gt.values)
            if lb[0] == 0:
                labels.append('Practitioner')
            if lb[0] == 1:
                labels.append('Promotional')
                
        
    #tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    tsne_model = TSNE(perplexity=50, n_components=2, init='pca', n_iter=2500, learning_rate = 200, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
    #print(labels) #array([1.]), array([1.])]
#     print('x', min(x),max(x))
#     print('y', min(y), max(y))

    plt.figure(figsize=(6, 6)) 
    
#     plt.xlim(-10, 10)
#     plt.ylim(-80, 50)
    import matplotlib
    classes = ['Practitioner', 'Promotional']
    #colors = ['green','red']
    #plt.scatter(x,y, c = labels , cmap= matplotlib.colors.ListedColormap(colors))
    import seaborn as sns
    sns.scatterplot(x=x, y=y, hue=labels)
    #plt.show()
    plt.savefig('data/keto/embedding_plot/graph_learn_keto.png')
tsne_plot(user_dic)
